<a href="https://colab.research.google.com/github/Rohanpk23/European-Soccer-Database/blob/master/European_Soccer_Database_(SQLALCHEMY_ORM_based).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1 :- 
1.   First connect to the google drive
2.   Make sure the sqlite file which will be used to retrive data is already stored in the google drive.
1.   Mount the drive using google drive stream

In [1]:
#Connecting to google drive for interacting with the sqlite database file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 2 :- Using the interface (engine) to connect with the sqlite database & listing all tables in the database file

In [2]:
# Engine is the common interface to connect to the local database
from sqlalchemy import create_engine, MetaData, Table, select, and_, or_, desc, func, case

# Importing the Python libraries 
import pandas as pd
import matplotlib.pyplot as plt

# Gets the metadata of the tables
metadata = MetaData() 

# Creating engine for the local file; sqlite:/// is the driver and football.sqite is the sqlite file
engine = create_engine('sqlite:////content/drive/My Drive/Colab Notebooks/football.sqlite')

# Connection established to access and manipulate data in database
connection = engine.connect()

Loading all the tables in schema

In [3]:
# All tables present in the database
print(engine.table_names())

['Country', 'League', 'Match', 'Player', 'Player_Attributes', 'Team', 'Team_Attributes', 'sqlite_sequence']


In [4]:
# Creating a reflection of the tables
player = Table('player', metadata, autoload = True, autoload_with = engine)
match = Table('match', metadata, autoload = True, autoload_with = engine)
team = Table('team', metadata, autoload = True, autoload_with = engine)

print(player.columns.keys())
print(repr(player))

['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'birthday', 'height', 'weight']
Table('player', MetaData(bind=None), Column('id', INTEGER(), table=<player>, primary_key=True, nullable=False), Column('player_api_id', INTEGER(), table=<player>), Column('player_name', TEXT(), table=<player>), Column('player_fifa_api_id', INTEGER(), table=<player>), Column('birthday', TEXT(), table=<player>), Column('height', INTEGER(), table=<player>), Column('weight', INTEGER(), table=<player>), schema=None)


A Warmup practice !!

In [14]:
# Stmt2 object has the select statement
stmt2 = select([player.columns.player_name, 
                player.columns.height, 
                player.columns.weight])

# Where clause with a combination of two conditions 
stmt2 = stmt2.where(
              and_(player.columns.height > 200, player.columns.weight > 200))

# Connection.execute() method on stmt2 object gives the ResultProxy object and fetchall() method gives the ResultSet object which is a list
results = connection.execute(stmt2).fetchmany(size = 50)

# Getting the records. Data is retrieved column wise for every tuple stored in records
for records in results:
  print(records.player_name, records.height, records.weight)

Abdoul Ba 200.66 212
Bogdan Milic 203.2 216
Costel Pantilimon 203.2 212
Jurgen Wevers 203.2 212
Kristof van Hout 208.28 243
Nikola Zigic 203.2 212
Pietro Marino 203.2 209
Stefan Maierhofer 203.2 216
Vanja Milinkovic-Savic 203.2 203
Wojciech Kaczmarek 200.66 218
Zeljko Kalac 203.2 209


In [6]:
# This retrieves tuple wise 
for records in results:
  print(results[:20]) 

[('Abdoul Ba',), ('Bogdan Milic',), ('Costel Pantilimon',), ('Jurgen Wevers',), ('Kristof van Hout',), ('Nikola Zigic',), ('Pietro Marino',), ('Stefan Maierhofer',), ('Vanja Milinkovic-Savic',), ('Wojciech Kaczmarek',), ('Zeljko Kalac',)]
[('Abdoul Ba',), ('Bogdan Milic',), ('Costel Pantilimon',), ('Jurgen Wevers',), ('Kristof van Hout',), ('Nikola Zigic',), ('Pietro Marino',), ('Stefan Maierhofer',), ('Vanja Milinkovic-Savic',), ('Wojciech Kaczmarek',), ('Zeljko Kalac',)]
[('Abdoul Ba',), ('Bogdan Milic',), ('Costel Pantilimon',), ('Jurgen Wevers',), ('Kristof van Hout',), ('Nikola Zigic',), ('Pietro Marino',), ('Stefan Maierhofer',), ('Vanja Milinkovic-Savic',), ('Wojciech Kaczmarek',), ('Zeljko Kalac',)]
[('Abdoul Ba',), ('Bogdan Milic',), ('Costel Pantilimon',), ('Jurgen Wevers',), ('Kristof van Hout',), ('Nikola Zigic',), ('Pietro Marino',), ('Stefan Maierhofer',), ('Vanja Milinkovic-Savic',), ('Wojciech Kaczmarek',), ('Zeljko Kalac',)]
[('Abdoul Ba',), ('Bogdan Milic',), ('Costel

Each season's count of :-

*   Goals scored by teams in their home grounds
*   Goals scored by teams in their overseas games



In [15]:
# SUM aggregation
Home_Tot = func.sum(match.columns.home_team_goal).label('Home Goals')
Away_Tot = func.sum(match.columns.away_team_goal).label('Away Goals')

# SUM aggregation included in SELECT clause
stmt3 = select([match.columns.season, Home_Tot, 
                Away_Tot, (Home_Tot - Away_Tot).label('Diff')])  

# GROUP BY clause
stmt3 = stmt3.group_by(match.columns.season)

# ORDER BY clause
stmt3_ordered= stmt3.order_by('Home Goals')   # Take care while ordering by an aliased columns

In [16]:
# Executing gives ResultSet Object (list containing tuples/rows)
results = connection.execute(stmt3_ordered).fetchall()

# Converting the ResultSet to a DataFrame
df3 = pd.DataFrame(results)
df3.columns = results[0].keys()
df3

,season,Home Goals,Away Goals,Diff
0,2013/2014,4787,3602,1185
1,2009/2010,4978,3654,1324
2,2008/2009,5007,3665,1342
3,2010/2011,5048,3701,1347
4,2012/2013,5053,3986,1067
5,2014/2015,5055,3842,1213
6,2011/2012,5064,3683,1381
7,2015/2016,5135,4027,1108


1. Teams performing best before their home and overseas crowds

1.1. SQL statements to retreive data 

In [17]:
# CASE statement
cased_H= case(
             [
              (match.c.home_team_goal > match.c.away_team_goal, 'Win'),
              (match.c.home_team_goal < match.c.away_team_goal, 'Defeat')
             ],
              else_ = 'Tie'
            ).label('Outcome')

# CASE statement included in the select clause
stmt_H = select([match.columns.season, 
                 match.columns.stage,
                 team.columns.team_long_name.label('Team'), 
                 match.columns.home_team_goal.label('Score'),
                 match.columns.away_team_goal.label('Opponent Score'),
                 cased_H])

# INNER JOIN CLAUSE
stmt_H_Teams = stmt_H.select_from(
                               match.join(team, match.c.home_team_api_id == team.c.team_api_id)
                              )  
# Sorting based on Score in descending order
stmt_H_Teams = stmt_H_Teams.order_by(desc('Score')) 

1.2. Execution and conversion to DataFrame

In [18]:
# Executing gives ResultSet Object (list containing tuples/rows)
result_H= connection.execute(stmt_H_Teams).fetchmany(size = 20)

# Converting the ResultSet to a DataFrame
df_H = pd.DataFrame(result_H)
df_H.columns = result_H[0].keys()
print('\n------------------------- Top 20 Home Perfomers ----------------------------\n')
df_H


------------------------- Top 20 Home Perfomers ----------------------------



,season,stage,Team,Score,Opponent Score,Outcome
0,2010/2011,10,PSV,10,0,Win
1,2015/2016,16,Real Madrid CF,10,2,Win
2,2009/2010,13,Tottenham Hotspur,9,1,Win
3,2012/2013,27,FC Bayern Munich,9,2,Win
4,2010/2011,11,Celtic,9,0,Win
5,2014/2015,29,Real Madrid CF,9,1,Win
6,2009/2010,38,Chelsea,8,0,Win
7,2011/2012,3,Manchester United,8,2,Win
8,2012/2013,18,Chelsea,8,0,Win
9,2014/2015,8,Southampton,8,0,Win


2. Teams performing best before an overseas crowd

2.1. SQL statements to retrieve data

In [11]:
# CASE statement
cased_A = case(
             [
              (match.c.away_team_goal > match.c.home_team_goal, 'Win'),
              (match.c.away_team_goal < match.c.home_team_goal, 'Loss')
             ],
              else_ = 'Tie'
            ).label('Outcome')

# CASE statement included in the select clause
stmt_A = select([match.columns.season, 
                 match.columns.stage,
                 team.columns.team_long_name.label('Team'), 
                 match.columns.away_team_goal.label('Score'),
                 match.columns.home_team_goal.label('Opponent Score'),
                 cased_A])

# INNER JOIN CLAUSE
stmt_A_Teams = stmt_A.select_from(
                               match.join(team, match.c.away_team_api_id == team.c.team_api_id)
                              )  

# Sorting based on Score in descending order
stmt_A_Teams = stmt_A_Teams.order_by(desc('Score')) 

2.2. Execution and conversion to DataFrame

In [12]:
# Executing gives ResultSet Object (list containing tuples/rows)
result_A = connection.execute(stmt_A_Teams).fetchmany(size = 20)

# Converting the ResultSet to a DataFrame
df_A = pd.DataFrame(result_A)
df_A.columns = result_A[0].keys()
print('\n------------------------- Top 20 Overseas Perfomers ------------------------\n')

df_A


------------------------- Top 20 Overseas Perfomers ------------------------



,season,stage,Team,Score,Opponent Score,Outcome
0,2015/2016,30,Paris Saint-Germain,9,0,Win
1,2010/2011,33,FC Bayern Munich,8,1,Win
2,2010/2011,12,FC Barcelona,8,0,Win
3,2014/2015,35,FC Barcelona,8,0,Win
4,2014/2015,4,Real Madrid CF,8,2,Win
5,2015/2016,34,FC Barcelona,8,0,Win
6,2012/2013,19,Club Brugge KV,7,1,Win
7,2014/2015,22,KV Kortrijk,7,1,Win
8,2014/2015,11,OGC Nice,7,2,Win
9,2013/2014,15,FC Bayern Munich,7,0,Win
